In [1]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm

TICKER = "SPY"

INTERVAL = "1h"
if INTERVAL == "1h":
    PERIOD = "730d"
else:
    PERIOD = "max"

SHIFT = 1 
LOOKBACK = 10000

STRATEGY = ["Volume"]

def get_data (ticker = TICKER, lookback = LOOKBACK, interval = INTERVAL, ):

    df = yf.download(ticker, interval = interval, auto_adjust=False, period = PERIOD)
    df.columns = df.columns.get_level_values(0)
    df = df.reset_index(drop=True)
    
    return df.iloc[-lookback:, :]

# Add Features
# Add Target 
def add_target(df, shift = SHIFT):

    # What is the Close price Shift days from now?
    df[f"Close + {shift}"] = df["Close"].shift(-shift)
    # Is the price of today smaller than teh price of SHIFT days from now?
    df["Target"] = (df[f"Close + {shift}"] > df["Close"]) * 1

    return df

# Generate Regression Output
def generate_regression_output(df, features=STRATEGY, target="Target"):

    subset = df[features + [target]].dropna()
    X = subset[features]
    y = subset[target]
    # X_with_const = sm.add_constant(X)
    # # sm.OLS: wie groß die Preisänderung (Target) in Prozent sein würde.
    # # sm.Logit:ob die Preisänderung positiv oder negativ sein wird.
    # model = sm.Logit(y, X_with_const).fit()
    
    # print(model.summary())

    return df


# Validate Regression Output

def main():

    df = get_data()
    df = add_target(df)
    df = generate_regression_output(df)

    return df 

df = main()
df


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume,Close + 1,Target
